In [1]:
!pip install transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=eb1186ddc26e8b2d187469ed5f1855964f5237c72b65b901509389f388e1bbc9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import numpy as np
import torch
from torch.optim import AdamW
from transformers import BertModel, BertTokenizer, BertForQuestionAnswering,get_linear_schedule_with_warmup, BertForSequenceClassification, AdamW, TFBertForSequenceClassification, BertConfig, BertTokenizer, TFBertModel
from sentence_transformers import SentenceTransformer, models, util, InputExample, losses
from scipy.stats import spearmanr
from sklearn.metrics import r2_score
from torch.utils.data import DataLoader, Dataset
import tensorflow as tf
from tqdm import tqdm
import transformers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data_analysis = pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_analysis.xlsx")
train_data_644= pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_614.xlsx")
train_data_translated = pd.read_excel("/kaggle/input/auto-grading-data/Traing_Data_translated.xlsx")
test_data = pd.read_excel("/kaggle/input/auto-grading-data/Test_Dataset.xlsx")

In [4]:
def Gradanizer(num, grade_mode="Fair"):
    """
    This function take the garde form 0 to 1 and output that garde in form 0 to 5

    parameters
    num: the garde in form 0 to 1
    grade_mode: how fair you want to model to be [fair: the exact transformation with out any lose ranges, easy: more skewed into higher grades more 
    common, lose_ends: more skewed into higher grades and lower grades]
    """
    def get_region_value_f(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364), (3.6364, 4.0909),
        (4.0909, 4.5455), (4.5455, 6)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
    def get_region_value_e(number):    
        intervals = [
        (-10, 0.4545), (0.4545, 0.9091), (0.9091, 1.3636),
        (1.3636, 1.8182), (1.8182, 2.2727), (2.2727, 2.7273),
        (2.7273, 3.1818), (3.1818, 3.6364)]
        values=[0,0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
    def get_region_value_l(number):    
        intervals = [
        (1.0000, 1.3571), (1.3571, 1.7143), (1.7143, 2.0714),
        (2.0714, 2.4286), (2.4286, 2.7857), (2.7857, 3.1429),
        (3.1429, 3.5000)]
        values=[0.5,1,1.5,2,2.5,3,3.5]
        for i, (start, end) in enumerate(intervals):
            if start <= number < end:
                return values[i]
    if (grade_mode == "fair"):
        num_f=num*5
        return get_region_value_f(num_f)
    elif (grade_mode == "easy"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        else:
            num_e = num*5
            return get_region_value_e(num_e)
    elif (grade_mode == "lose_ends"):
        if (num >=0.85):
            return 5
        elif ( 0.85>num>=0.8):
            return 4.5
        elif ( 0.8>num>=0.7):
            return 4
        elif (0.2>=num):
            return 0
        else:
            num_l = num*5
            return get_region_value_l(num_l)
    else:
        print("not a valid mode ")

In [5]:
def Evaluation(embeddings_1=None, embeddings_2=None,test_data=None,mode="easy"):
    """
    this function takes two lists of embeddings in the form of torch tensor or list or df column one for the student answer and other for model answer
    or any two sentences if this function used in any other code and the test data frame and the mode of Gradazier and output MAE % and r_squared and 
    correlation_coefficient

    parameters
    embeddings_1:list or tensor or df column that hold the sentence embeddings (student answer embeddings)
    embeddings_2:list or tensor or df column that hold the sentence embeddings (model answer embeddings)
    data: the test data frame
    mode: the mode of which gradazier is used ["fair", "easy", "lose_ends"]
    """
    from sentence_transformers import util
    if (len(embeddings_1) != len(embeddings_2)):
        print("embeddings_1 and embeddings_2 are not the same length")
        return None
    elif (mode not in ["easy","lose_ends","fair"]):
        print("not valid mode")
        return None
    else:
        predicted = []
        for i in range(len(embeddings_1)):
            predicted.append(Gradanizer(util.cos_sim(embeddings_1[i], embeddings_2[i]),mode))
        transformed_grade=[]
        for i in range(len(embeddings_1)):
            #transformed_grade.append(Gradanizer(test_data["grade"][i],mode))
            transformed_grade.append(test_data["grade"][i]*5)
        arr_predicted =np.array(predicted)
        arr_grade=np.array(transformed_grade)
        MAE = (1-((np.sum(np.abs(arr_predicted-arr_grade))/len(embeddings_1))/5))*100
        correlation_coefficient, p_value = spearmanr(arr_grade, arr_predicted)
        r_squared = r2_score(arr_grade, arr_predicted)
        dif =arr_predicted-arr_grade
        hits_precentage= (np.count_nonzero(dif == 0)/len(embeddings_1))
        #return MAE,correlation_coefficient,r_squared
        #if you want the hit precentage tag the above and untag the blew
        return MAE,correlation_coefficient,hits_precentage
    

In [6]:
# Load pre-trained BERT model and tokenizer for question-answering

tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

# Assuming your train_data is a DataFrame with 'student_answer', 'model_answer', and 'grade' columns
train_data = train_data_analysis

class QADataset(Dataset):
    def __init__(self, tokenizer, data, max_length=256):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        student_answer = str(self.data.iloc[idx]['student_answer'])
        model_answer = str(self.data.iloc[idx]['model_answer'])
        grade = str(self.data.iloc[idx]['grade'])

        # Tokenize and encode the inputs
        inputs = tokenizer(
            student_answer,
            model_answer,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation="only_first",
            return_overflowing_tokens=True,
            return_token_type_ids=True,
        )

        # Assuming 'grade' is your target variable
        label = torch.tensor(int(float(self.data.iloc[idx]['grade'])))  # Convert to int

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': label,
            'end_positions': label
        }

# Create a DataLoader for training
train_dataset = QADataset(tokenizer, train_data)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader))

# Training loop
num_epochs = 3 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}"):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'start_positions': batch['start_positions'].to(device),
            'end_positions': batch['end_positions'].to(device)
        }

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

...

Epoch 1, Average Loss: 0.6865524751610226


Epoch 2: 100%|██████████| 225/225 [01:25<00:00,  2.64it/s]


Epoch 2, Average Loss: 0.5260076000955369


Epoch 3: 100%|██████████| 225/225 [01:28<00:00,  2.55it/s]

Epoch 3, Average Loss: 0.5289428808954026


In [7]:
def calculate_embeddings(texts):
    model.eval()
    embeddings = []

    for text in texts:
        # Tokenize and encode the input text
        inputs = tokenizer(text, return_tensors='pt', max_length=256, truncation=True, padding=True)
        inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the same device as the model

        # Ensure the model returns hidden states
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)

        # Extract embeddings from a specific layer (adjust layer_index as needed)
        layer_index = 11  # You may need to adjust this based on your model architecture
        hidden_states = outputs.hidden_states[layer_index]

        # Use the [CLS] token representation (index 0) as the embedding
        embeddings.append(hidden_states[:, 0, :].cpu().numpy())

    return embeddings



# Calculate embeddings for student answers
student_embeddings = calculate_embeddings(test_data['student_answer'].tolist())

# Calculate embeddings for model answers
model_embeddings = calculate_embeddings(test_data['model_answer'].tolist())


In [8]:
student_embeddings[0]

array([[ 4.09536138e-02,  3.29970032e-01, -4.38342482e-01,
        -1.96466833e-01, -5.53528845e-01, -6.75534129e-01,
        -1.97541252e-01,  3.10482055e-01, -2.64965385e-01,
        -8.49555656e-02, -1.20658040e+00, -3.11738193e-01,
        -6.60098016e-01,  6.90741062e-01, -9.06425893e-01,
        -1.14930725e+00,  9.74403441e-01, -7.78743625e-01,
         8.10871422e-01, -3.53649050e-01,  8.10040951e-01,
         1.92555517e-01, -1.40749544e-01, -1.00594974e+00,
        -2.65151560e-01, -8.26197505e-01,  1.06149673e+00,
        -2.86621600e-01,  9.46839631e-01,  5.16767800e-01,
        -2.09790006e-01,  1.17275035e+00, -8.24627161e-01,
        -6.41236722e-01,  6.12398148e-01,  1.50120482e-01,
        -2.53612339e-01, -7.22801447e-01, -2.35575527e-01,
         9.28565800e-01,  4.99484867e-01,  7.68778265e-01,
         6.05363138e-02, -3.13230425e-01,  9.13457334e-01,
        -5.42600513e-01,  2.75858045e-02,  6.03766024e-01,
         4.62974161e-01,  6.35824561e-01,  2.10252970e-0

In [9]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

72.04665959703075 0.12573215347203928 0.2354188759278897


In [10]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

70.69989395546129 -0.0454614656723628 0.2672322375397667


In [11]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

70.71049840933192 -0.045426900481152936 0.2672322375397667


In [12]:
# Assuming you have a new dataset for training, replace this with your actual new data
new_train_data = train_data_644

# Custom dataset class for question-answering
class NewQADataset(Dataset):
    def __init__(self, tokenizer, data, max_length=256):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        student_answer = str(self.data.iloc[idx]['student_answer'])
        model_answer = str(self.data.iloc[idx]['model_answer'])
        grade = str(self.data.iloc[idx]['grade'])

        # Tokenize and encode the inputs
        inputs = tokenizer(
            student_answer,
            model_answer,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation="only_first",
            return_overflowing_tokens=True,
            return_token_type_ids=True,
        )


        # Assuming 'grade' is your target variable
        label = torch.tensor(int(float(self.data.iloc[idx]['grade'])))  # Convert to int

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': label,
            'end_positions': label
        }

# Create a DataLoader for training on the new data
new_train_dataset = NewQADataset(tokenizer, new_train_data)
new_train_loader = DataLoader(new_train_dataset, batch_size=8, shuffle=True)

# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(new_train_loader))

# Training loop on the new data
num_epochs = 3 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(new_train_loader, desc=f"Epoch {epoch + 1}"):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'start_positions': batch['start_positions'].to(device),
            'end_positions': batch['end_positions'].to(device)
        }

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    average_loss = total_loss / len(new_train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")

Epoch 1: 100%|██████████| 34/34 [00:13<00:00,  2.43it/s]


Epoch 1, Average Loss: 0.6377492836293053


Epoch 2: 100%|██████████| 34/34 [00:13<00:00,  2.43it/s]


Epoch 2, Average Loss: 0.5008435117847779


Epoch 3: 100%|██████████| 34/34 [00:14<00:00,  2.42it/s]

Epoch 3, Average Loss: 0.4940608792445239


In [13]:
# Calculate embeddings for student answers
student_embeddings1 = calculate_embeddings(test_data['student_answer'].tolist())

# Calculate embeddings for model answers
model_embeddings1 = calculate_embeddings(test_data['model_answer'].tolist())


In [14]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings1,model_embeddings1,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

71.66489925768823 0.10386830689417545 0.24602332979851538


In [15]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings1,model_embeddings1,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

70.61505832449629 -0.0626258505790398 0.27465535524920465


In [16]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings1,model_embeddings1,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

70.63626723223754 -0.0626258505790398 0.27465535524920465


In [17]:
# Assuming you have a new dataset for training, replace this with your actual new data
new_train_data = train_data_translated

# Custom dataset class for question-answering
class NewQADataset(Dataset):
    def __init__(self, tokenizer, data, max_length=256):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        student_answer = str(self.data.iloc[idx]['student_answer'])
        model_answer = str(self.data.iloc[idx]['model_answer'])
        grade = str(self.data.iloc[idx]['grade'])

        # Tokenize and encode the inputs
        inputs = tokenizer(
            student_answer,
            model_answer,
            return_tensors='pt',
            max_length=self.max_length,
            padding='max_length',
            truncation="only_first",
            return_overflowing_tokens=True,
            return_token_type_ids=True,
        )

        # Assuming 'grade' is your target variable
        label = torch.tensor(int(float(self.data.iloc[idx]['grade'])))  # Convert to int

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'start_positions': label,
            'end_positions': label
        }

# Create a DataLoader for training on the new data
new_train_dataset = NewQADataset(tokenizer, new_train_data)
new_train_loader = DataLoader(new_train_dataset, batch_size=8, shuffle=True)

# Define optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(new_train_loader))

# Training loop on the new data
num_epochs = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(new_train_loader, desc=f"Epoch {epoch + 1}"):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device),
            'start_positions': batch['start_positions'].to(device),
            'end_positions': batch['end_positions'].to(device)
        }

        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    average_loss = total_loss / len(new_train_loader)
    print(f"Epoch {epoch + 1}, Average Loss: {average_loss}")


Epoch 1: 100%|██████████| 213/213 [01:25<00:00,  2.50it/s]


Epoch 1, Average Loss: 0.30604616013473607


Epoch 2: 100%|██████████| 213/213 [01:26<00:00,  2.47it/s]


Epoch 2, Average Loss: 0.22752694029168624


Epoch 3: 100%|██████████| 213/213 [01:26<00:00,  2.47it/s]

Epoch 3, Average Loss: 0.2279118604503327


In [18]:
# Calculate embeddings for student answers
student_embeddings2 = calculate_embeddings(test_data['student_answer'].tolist())

# Calculate embeddings for model answers
model_embeddings2 = calculate_embeddings(test_data['model_answer'].tolist())


In [19]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

71.41039236479321 0.055040866462192084 0.2449628844114528


In [20]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

70.59384941675503 -0.057739387566184235 0.2788971367974549


In [21]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

70.59384941675503 -0.057739387566184235 0.2788971367974549


## BERT by Sentence Transformers

In [22]:
def data_loader_local(data=None):
    input_data = []
    for index, row in data.iterrows():
        student_answer = row['student_answer']
        model_answer = row['model_answer']
        grade = row['grade']
    
        # Create InputExample instances
        input_data.append(InputExample(texts=[student_answer, model_answer], label=grade))
    return(input_data)

In [23]:
# Get train_data_analysis DataFrame
train_data = data_loader_local(train_data_analysis)

In [24]:
# Define the model
Model_bert = SentenceTransformer('bert-base-nli-mean-tokens')

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [25]:
# Define a data loader
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

In [26]:
# Define a loss function (e.g., cosine similarity loss)
loss_function = losses.CosineSimilarityLoss(model=Model_bert)

In [27]:
# Fine-tune the model
Model_bert.fit(
    train_objectives=[(train_dataloader, loss_function)],
    epochs=5,
    warmup_steps=200,
    show_progress_bar=True
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

Iteration:   0%|          | 0/113 [00:00<?, ?it/s]

In [28]:
student_embeddings=Model_bert.encode(test_data["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [29]:
model_embeddings=Model_bert.encode(test_data["model_answer"].to_list(), convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [30]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

80.92258748674443 0.5588385970379385 0.26935312831389185


In [31]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

79.81972428419937 0.5245996241138485 0.3213149522799576


In [32]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings,model_embeddings,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

79.80911983032874 0.5252505888859174 0.32343584305408274


In [33]:
# Get train_data_644 DataFrame
train_data644 = data_loader_local(train_data_644)

In [34]:
# Define a data loader
train_dataloader644 = DataLoader(train_data644, batch_size=16, shuffle=True)

In [35]:
# Define a loss function (e.g., cosine similarity loss)
loss_function644 = losses.CosineSimilarityLoss(model=Model_bert)

In [36]:
# Fine-tune the model
Model_bert.fit(
    train_objectives=[(train_dataloader644, loss_function644)],
    epochs=5,
    warmup_steps=200,
    show_progress_bar=True
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

In [37]:
student_embeddings2=Model_bert.encode(test_data["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [38]:
model_embeddings2=Model_bert.encode(test_data["model_answer"].to_list(), convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [39]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

81.47401908801697 0.5488868599018725 0.22375397667020147


In [40]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

81.12407211028632 0.5251614090357453 0.2788971367974549


In [41]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings2,model_embeddings2,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

80.72110286320255 0.5260094588684475 0.27465535524920465


In [42]:
# Get train_data_translated DataFrame
train_datatr = data_loader_local(train_data_translated)

In [43]:
# Define a data loader
train_dataloadertr = DataLoader(train_datatr, batch_size=16, shuffle=True)

In [44]:
# Define a loss function (e.g., cosine similarity loss)
loss_functiontr = losses.CosineSimilarityLoss(model=Model_bert)

In [45]:
# Fine-tune the model
Model_bert.fit(
    train_objectives=[(train_dataloadertr, loss_functiontr)],
    epochs=5,
    warmup_steps=200,
    show_progress_bar=True
)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

Iteration:   0%|          | 0/107 [00:00<?, ?it/s]

In [46]:
student_embeddings3=Model_bert.encode(test_data["student_answer"].to_list(),convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [47]:
model_embeddings3=Model_bert.encode(test_data["model_answer"].to_list(), convert_to_tensor=True)

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [48]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings3,model_embeddings3,test_data,"fair")
print(precentage,correlation_coefficient,hits_precentage)

83.17073170731707 0.6699247879360674 0.20996818663838812


In [49]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings3,model_embeddings3,test_data,"easy")
print(precentage,correlation_coefficient,hits_precentage)

83.56309650053022 0.6671765502998961 0.2704135737009544


In [50]:
precentage,correlation_coefficient,hits_precentage = Evaluation(student_embeddings3,model_embeddings3,test_data,"lose_ends")
print(precentage,correlation_coefficient,hits_precentage)

82.50265111346766 0.6660973496643915 0.264050901378579
